In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns
import numpy as np

In [ ]:
sns.set_style('ticks')

In [ ]:
def demand_binomial_tree(n = 6, p_up = 0.3, p_down = 0.7, inc=2200/2093, dinc=2093/2200, v0 = 2200):
    """
        From Wikipedia: The binomial options pricing model (BOPM) provides a generalizable
        numerical method for the valuation of options. Essentially, the model uses a
        "discrete-time" (lattice based) model of the varying price over time of
        the underlying financial instrument. It was proposed by Sharpe (1978), and formalized
        Cox et al. (1979) and Rendleman & Bartter (1979).
        
        This BOPM gives the demand and associated probability for the Option Games paper
        
        Parameters
        ----------
        n: Number of time steps on the tree (e.g. years) (int)
        
        Returns
        -------
        G: A graph with demand associated probabilities.
    """

    G = nx.DiGraph()
    for i in range(0, n+1):
        for j in range(1, i+2):
            if i < n:
                G.add_edge((i,j),(i+1,j), weight=p_up, inc=inc)
                G.add_edge((i,j),(i+1,j+1), weight=p_down,  inc=dinc)

    for node in G.nodes():
        G.nodes[node]['pos']=(node[0],n+2+node[0]-2*node[1])
        G.nodes[node]['demand'] = 0
        G.nodes[node]['prob'] = 0

    G.nodes[0,1]['demand'] = v0
    G.nodes[0,1]['prob'] = 1

    for node in G.nodes():
        for succ in G.successors(node):
            # Edge weight
            i = G.get_edge_data(node, succ)['inc']
            d = G.nodes[node]['demand']
            p = G.nodes[node]['prob']

            w = G.get_edge_data(node, succ)['weight']

            G.nodes[succ]['demand'] = i * d
            G.nodes[succ]['prob'] += w * p
            
    return G


In [ ]:
G = demand_binomial_tree()

In [ ]:
figsize = (15, 7)

In [ ]:
plt.figure(figsize=figsize)
plt.title('Demand evolution')

labels = nx.get_node_attributes(G, 'demand')
pos=nx.get_node_attributes(G,'pos')

for l in labels:
    labels[l] = np.int(np.round(labels[l], 0))

nx.draw(G, pos, labels=labels, node_size=2000)

labels = nx.get_edge_attributes(G,'inc')
for l in labels:
    labels[l] = '{:.1f}%'.format((1 - labels[l])*100)

nx.draw_networkx_edge_labels(G,pos,edge_labels=labels);

In [ ]:
plt.figure(figsize=figsize)
plt.title('Probability of demand evolution')

labels = nx.get_node_attributes(G, 'prob')
pos=nx.get_node_attributes(G,'pos')

for l in labels:
    labels[l] = '{:.1f}%'.format(labels[l]*100)

nx.draw(G, pos, labels=labels, node_size=2000)

labels = nx.get_edge_attributes(G,'weight')

for l in labels:
    labels[l] = '{:.0f}%'.format(labels[l]*100)
    
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels);

## Example projects

In [ ]:
projects = {
    'MineCo' : {
        'CAPEX' : 250,
        'OPEX' : 687,
        'Capacity' : 250e3
    },
    'CompCo' : {
        'CAPEX' : 150,
        'OPEX' : 740,
        'Capacity' : 320e3
    }
}

projects

In [ ]:
t = 20 # Years
t0 = 3 # Years

price = 1000 # USD/ton

fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=figsize)
# Remove horizontal space between axes
fig.subplots_adjust(hspace=0, wspace=0)
ax[0].set_title('Nominal cash flow')

for i, comp in enumerate(projects):
    CAPEX = np.zeros(t)
    OPEX = np.zeros(t)
    revenues = np.zeros(t)

    CAPEX[0:3] = -projects[comp]['Capacity'] * projects['MineCo']['CAPEX']/3. / 1e6
    OPEX[3:] = -projects[comp]['Capacity'] * projects['MineCo']['OPEX'] / 1e6

    revenues[3:] = projects[comp]['Capacity'] * price/1e6


    ax[i].bar(range(t), CAPEX, label='CAPEX', width=1)
    ax[i].bar(range(t), OPEX, label='OPEX', width=1)
    ax[i].bar(range(t), revenues, label='Revenues', width=1)
    ax[i].plot(CAPEX + OPEX + revenues, 'r-o', label='net cash flow')

    ax[i].set_ylabel('{} CF (MUSD)'.format(comp))

    ax[i].legend()
    
ax[1].set_xlabel('years');

In [ ]:
d = np.arange(t)
d = 1/np.power(1.05, d)

fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=figsize)
# Remove horizontal space between axes
fig.subplots_adjust(hspace=0, wspace=0)
ax[0].set_title('Discounted cash flow')

for i, comp in enumerate(projects):
    CAPEX = np.zeros(t)
    OPEX = np.zeros(t)
    revenues = np.zeros(t)

    CAPEX[0:3] = -projects[comp]['Capacity'] * projects['MineCo']['CAPEX']/3. / 1e6
    OPEX[3:] = -projects[comp]['Capacity'] * projects['MineCo']['OPEX'] / 1e6

    revenues[3:] = projects[comp]['Capacity'] * price/1e6


    ax[i].bar(range(t), CAPEX*d, label='CAPEX', width=1)
    ax[i].bar(range(t), OPEX*d, label='OPEX', width=1)
    ax[i].bar(range(t), revenues*d, label='Revenues', width=1)
    ax[i].plot((CAPEX + OPEX + revenues)*d, 'r-o', label='net cash flow')

    ax[i].set_ylabel('{} DCF (MUSD)'.format(comp))

    ax[i].legend(loc='upper left')
    
    print(comp, np.sum((CAPEX + OPEX + revenues)*d).round(2))
    
ax[1].set_xlabel('years');

In [ ]:
demand = [1630, 1713, 1801, 1894, 1991, 2093, 2200, 2313, 2431, 2556, 2687, 2825, 2970]
price = [680, 685, 685, 685, 687, 700, 700, 700, 740, 1000, 1000, 1000, 1000]

plt.figure(figsize=figsize)
plt.step(demand, price, where='post')
plt.xlabel('Demand (kt)')
plt.ylabel('Price (USD/kt)')

## References

Cox, J. C.; Ross, S. A.; Rubinstein, M. (1979). "Option pricing: A simplified approach". Journal of Financial Economics. 7 (3): 229. CiteSeerX 10.1.1.379.7582. doi:10.1016/0304-405X(79)90015-1.

Richard J. Rendleman, Jr. and Brit J. Bartter. 1979. "Two-State Option Pricing". Journal of Finance 24: 1093-1110. doi:10.2307/2327237